In [3]:
import os
from pathlib import Path

import pandas as pd
from sklearn.model_selection import StratifiedGroupKFold

In [36]:
def get_participant_data(root_dir):
    # Load and preprocess data
    data = []
    root = Path(root_dir)
    for participant_dir in root.glob("*"):
        if not participant_dir.is_dir():
            continue
        participant_id = participant_dir.name
        for video_file in participant_dir.glob("*.npz"):
            filename_parts = video_file.stem.split("_")
            label = int(filename_parts[2])
            # if label != 2:  # Ignore neutral class
            binary_label = 1 if label > 2 else 0
            data.append(
                {
                    "ParticipantId": participant_id,
                    "Label": label,
                    "X": video_file
                }
            )
    return pd.DataFrame(data)

In [37]:
df = get_participant_data("/media/ammar/New Volume/Engagement Data/Features")
df

,ParticipantId,Label,X
0,118,0,/media/ammar/New Volume/Engagement Data/Featur...
1,118,4,/media/ammar/New Volume/Engagement Data/Featur...
2,118,0,/media/ammar/New Volume/Engagement Data/Featur...
3,118,0,/media/ammar/New Volume/Engagement Data/Featur...
4,118,0,/media/ammar/New Volume/Engagement Data/Featur...
...,...,...,...
868,458,3,/media/ammar/New Volume/Engagement Data/Featur...
869,458,4,/media/ammar/New Volume/Engagement Data/Featur...
870,458,2,/media/ammar/New Volume/Engagement Data/Featur...
871,458,3,/media/ammar/New Volume/Engagement Data/Featur...


In [38]:
sgkf = StratifiedGroupKFold(n_splits=7, shuffle=True, random_state=42)
for i, (dev_index, test_index) in enumerate(sgkf.split(df, df.Label, df.ParticipantId)):
    dev_df = df.iloc[dev_index]
    test_df = df.iloc[test_index]
    for j, (train_index, val_index) in enumerate(sgkf.split(dev_df, dev_df.Label, dev_df.ParticipantId)):
        train_df = dev_df.iloc[train_index]
        val_df = dev_df.iloc[val_index]
        break
    break

In [39]:
val_df.ParticipantId.nunique()

5

In [42]:
train_df[["ParticipantId"]].drop_duplicates()

,ParticipantId
27,183
38,804
53,297
81,559
99,291
120,540
159,623
183,568
206,444
254,478
